In [1]:
import time
import pandas as pd
import tushare as ts
from sqlalchemy import create_engine, text
from tqdm import tqdm

# 导入您的配置文件
try:
    import config
except ImportError:
    print("错误：无法找到配置文件 config.py。")
    exit()

# --- 新增：批处理相关配置 ---
BATCH_SIZE = 100 # 每获取100家公司的数据就保存一次
TARGET_TABLE_NAME = 'stock_financial_indicators' # 目标数据表名

def save_batch_to_db(data_list, db_engine):
    """将一个批次的数据保存到数据库"""
    if not data_list:
        return
    try:
        batch_df = pd.concat(data_list, ignore_index=True)
        # 使用 'append' 模式追加数据
        batch_df.to_sql(TARGET_TABLE_NAME, db_engine, if_exists='append', index=False)
        # print(f"\n成功保存一个批次（{len(data_list)}只股票）的数据到数据库。")
    except Exception as e:
        print(f"\n保存批次数据到数据库时出错: {e}")

def fetch_all_financial_data_resumable():
    """
    (可断点续传版)
    连接数据库，获取所有股票代码，并循环调用Tushare API获取财务指标数据。
    """
    # 1. 初始化和连接
    try:
        pro = ts.pro_api(config.TUSHARE_TOKEN)
        db_uri = (
            f"postgresql+psycopg2://{config.DB_USER}:{config.DB_PASS}@"
            f"{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"
        )
        engine = create_engine(db_uri)
        print("Tushare API 和数据库连接初始化成功。")
    except Exception as e:
        print(f"初始化连接时发生错误: {e}")
        return

    # 2. --- 断点续传逻辑 ---
    processed_codes = set()
    try:
        # 检查目标表是否存在，并读取已处理的股票代码
        with engine.connect() as connection:
            if engine.dialect.has_table(connection, TARGET_TABLE_NAME):
                processed_df = pd.read_sql(f"SELECT DISTINCT ts_code FROM {TARGET_TABLE_NAME}", engine)
                processed_codes = set(processed_df['ts_code'])
                print(f"检测到目标表，已处理过 {len(processed_codes)} 只股票。")
    except Exception as e:
        print(f"检查已处理股票时出错（如果表是首次创建，此信息可忽略）: {e}")

    # 3. 获取全部待处理的股票列表，并排除已处理的
    try:
        all_stocks_df = pd.read_sql("SELECT ts_code FROM stock_basic_info", engine)
        full_stock_list = all_stocks_df['ts_code'].tolist()
        
        # 筛选出未处理的股票代码
        codes_to_process = [code for code in full_stock_list if code not in processed_codes]

        if not codes_to_process:
            print("所有股票数据均已获取，无需执行任务。")
            return
        
        print(f"共 {len(full_stock_list)} 只股票，还需处理 {len(codes_to_process)} 只。")

    except Exception as e:
        print(f"从数据库读取股票列表时出错: {e}")
        return

    # 4. --- 批处理循环 ---
    batch_data = []
    start_date = '20200101'
    end_date = '20241231'

    print(f"开始获取 {start_date} 至 {end_date} 的财务指标数据...")

    for ts_code in tqdm(codes_to_process, desc="正在获取财务指标"):
        try:
            df = pro.fina_indicator(ts_code=ts_code, start_date=start_date, end_date=end_date)
            if not df.empty:
                batch_data.append(df)
            
            # 检查是否达到批处理大小
            if len(batch_data) >= BATCH_SIZE:
                save_batch_to_db(batch_data, engine)
                batch_data = [] # 清空批次列表，准备下一批

            time.sleep(0.7) # 保持安全的请求间隔

        except Exception as e:
            print(f"\n获取 {ts_code} 数据时出错: {e}。跳过该股票。")
            continue

    # 5. 处理并保存最后一个批次（可能不足BATCH_SIZE）
    if batch_data:
        print("\n正在保存最后一个批次的数据...")
        save_batch_to_db(batch_data, engine)

    print("\n所有剩余股票数据处理完毕！")


if __name__ == '__main__':
    fetch_all_financial_data_resumable()

Tushare API 和数据库连接初始化成功。
共 5413 只股票，还需处理 5413 只。
开始获取 20200101 至 20241231 的财务指标数据...


正在获取财务指标:   2%|▍                    | 99/5413 [01:34<1:23:39,  1.06it/s]/var/folders/jk/_51b3nb118q00wt960d15lz00000gn/T/ipykernel_56081/1053922729.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  batch_df = pd.concat(data_list, ignore_index=True)
正在获取财务指标:  79%|███████████████    | 4300/5413 [1:11:09<18:13,  1.02it/s]/var/folders/jk/_51b3nb118q00wt960d15lz00000gn/T/ipykernel_56081/1053922729.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  batch_df = pd.concat(data_list, ignore_index=Tr


正在保存最后一个批次的数据...

所有剩余股票数据处理完毕！


In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import sys

try:
    # 导入我们的配置文件
    import config 
except ImportError:
    print("错误：无法找到配置文件 config.py。请确保该文件存在于同一目录下。")
    sys.exit(1)

def view_top_10_financials():
    """
    连接数据库并打印 stock_financial_indicators 表的前10行内容。
    """
    print("正在尝试连接数据库并查询数据...")
    
    try:
        # 1. 使用 config.py 中的配置信息构建数据库连接URI
        db_uri = (
            f"postgresql+psycopg2://{config.DB_USER}:{config.DB_PASS}@"
            f"{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"
        )
        engine = create_engine(db_uri)

        table_name = "stock_financial_indicators"
        
        # 2. 构建SQL查询语句，使用 LIMIT 10 来获取前10行
        query = text(f"SELECT * FROM {table_name} LIMIT 10;")
        
        # 3. 使用pandas执行查询并加载数据
        # read_sql 是一个非常方便的函数，能将SQL查询结果直接装入DataFrame
        df = pd.read_sql(query, engine)

        print(f"\n✅ 成功查询到 '{table_name}' 表的前10行数据：")
        
        # 4. 为了在终端中获得更好的显示效果，我们设置一下pandas的显示选项
        pd.set_option('display.max_columns', None) # 显示所有列，不省略
        pd.set_option('display.width', 120)       # 设置每行的显示宽度
        
        print(df)

    except Exception as e:
        print(f"\n❌ 操作失败，发生错误: {e}")

if __name__ == '__main__':
    view_top_10_financials()

正在尝试连接数据库并查询数据...

✅ 成功查询到 'stock_financial_indicators' 表的前10行数据：
     ts_code  ann_date  end_date   eps  dt_eps  total_revenue_ps  revenue_ps  capital_rese_ps  surplus_rese_ps  \
0  000001.SZ  20250315  20241231  2.15    2.15            7.5593      7.5593           4.1592           0.5556   
1  000001.SZ  20241019  20240930  1.94    1.94            5.7499      5.7499           4.1593           0.5556   
2  000001.SZ  20240816  20240630  1.23    1.23            3.9747      3.9747           4.1617           0.5556   
3  000001.SZ  20240816  20240630   NaN     NaN            3.9747      3.9747           4.1617           0.5556   
4  000001.SZ  20240420  20240331  0.66    0.66            1.9978      1.9978           4.1617           0.5556   
5  000001.SZ  20240315  20231231  2.25    2.25            8.4871      8.4871           4.1617           0.5556   
6  000001.SZ  20231025  20230930  1.94    1.94            6.5771      6.5771           4.1616           0.5556   
7  000001.SZ  20230824

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
import sys

try:
    import config 
except ImportError:
    print("错误：无法找到配置文件 config.py。")
    sys.exit(1)

def get_table_dimensions():
    """
    连接数据库，高效地获取 stock_financial_indicators 表的行数和列数。
    """
    print("正在连接数据库并获取表的维度...")
    
    try:
        db_uri = f"postgresql+psycopg2://{config.DB_USER}:{config.DB_PASS}@{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"
        engine = create_engine(db_uri)

        table_name = "stock_financial_indicators"
        
        # 1. 高效地获取总行数
        # COUNT(*) 是数据库最优化的操作之一，它只返回一个数字，不传输任何数据行
        row_count_query = text(f"SELECT COUNT(*) FROM {table_name};")
        with engine.connect() as connection:
            row_count = connection.execute(row_count_query).scalar_one()

        # 2. 高效地获取列数
        # 我们只需读取一行数据即可知道有多少列，无需加载整个表
        column_count_query = text(f"SELECT * FROM {table_name} LIMIT 1;")
        df_sample = pd.read_sql(column_count_query, engine)
        column_count = df_sample.shape[1]

        print("\n" + "="*40)
        print(f"查询结果:")
        print(f"  表名: {table_name}")
        print(f"  总行数: {row_count}")
        print(f"  总列数: {column_count}")
        print("="*40)

    except Exception as e:
        print(f"\n❌ 操作失败，发生错误: {e}")

if __name__ == '__main__':
    get_table_dimensions()

正在连接数据库并获取表的维度...

查询结果:
  表名: stock_financial_indicators
  总行数: 173411
  总列数: 108


In [4]:
import pandas as pd
from sqlalchemy import create_engine
import sys
import time

# 修正1：导入我们自己的工具和配置
try:
    import config 
    from utils import setup_logging
except ImportError:
    print("错误：无法找到配置文件 config.py 或工具文件 utils.py。")
    sys.exit(1)

def explore_financial_data():
    """
    (修正版) 连接数据库，加载财务指标数据，并输出一份全面的探索性分析报告，同时记录日志。
    """
    # 修正2：在函数开始时初始化日志记录器
    logger = setup_logging('data_exploration.log') # 为本次任务指定一个专门的日志文件名
    
    logger.info("================== 数据探索任务开始 ==================")
    logger.info("正在连接数据库...")
    
    try:
        db_uri = (
            f"postgresql+psycopg2://{config.DB_USER}:{config.DB_PASS}@"
            f"{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"
        )
        engine = create_engine(db_uri)
        table_name = "stock_financial_indicators"
        
        logger.info(f"开始从数据库加载 '{table_name}' 表...")
        start_time = time.time()
        df = pd.read_sql_table(table_name, engine)
        end_time = time.time()
        
        # 修正3：使用 logger 记录过程信息
        logger.info(f"数据加载完毕！共加载 {len(df)} 行数据，耗时 {end_time - start_time:.2f} 秒。")

        # --- 生成体检报告（保留print以获得最佳屏幕显示效果） ---
        
        # 1. 基本信息
        print("\n" + "="*50)
        print("--- 1. 数据基本信息 (df.info()) ---")
        print("="*50)
        df.info(verbose=True, show_counts=True)

        # 2. 数值列统计摘要
        print("\n" + "="*50)
        print("--- 2. 数值列统计摘要 (df.describe()) ---")
        print("="*50)
        pd.set_option('display.float_format', '{:.2f}'.format)
        print(df.describe().T)

        # 3. 各列缺失值比例
        print("\n" + "="*50)
        print("--- 3. 各列缺失值比例 (Missing Value %) ---")
        print("="*50)
        missing_percentage = (df.isnull().sum() / len(df)) * 100
        missing_df = missing_percentage.to_frame('missing_percentage').sort_values(by='missing_percentage', ascending=False)
        print(missing_df[missing_df['missing_percentage'] > 0])
        
        logger.info("体检报告生成完毕！")
        logger.info("================== 数据探索任务成功结束 ==================\n")

    except Exception as e:
        # 修正3：使用 logger 记录错误，exc_info=True会包含详细的错误堆栈信息
        logger.error("操作失败，发生严重错误: %s", e, exc_info=True)
        print(f"\n❌ 操作失败，发生错误，详情请查看日志文件。")

if __name__ == '__main__':
    explore_financial_data()

2025-06-11 09:20:46 - MyQuantProject - INFO - ================== 数据探索任务开始 ==================
2025-06-11 09:20:46 - MyQuantProject - INFO - 正在连接数据库...
2025-06-11 09:20:46 - MyQuantProject - INFO - 开始从数据库加载 'stock_financial_indicators' 表...
2025-06-11 09:20:51 - MyQuantProject - INFO - 数据加载完毕！共加载 173411 行数据，耗时 4.96 秒。



--- 1. 数据基本信息 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173411 entries, 0 to 173410
Data columns (total 108 columns):
 #    Column                    Non-Null Count   Dtype  
---   ------                    --------------   -----  
 0    ts_code                   173411 non-null  object 
 1    ann_date                  173335 non-null  object 
 2    end_date                  173411 non-null  object 
 3    eps                       168658 non-null  float64
 4    dt_eps                    164950 non-null  float64
 5    total_revenue_ps          168603 non-null  float64
 6    revenue_ps                168570 non-null  float64
 7    capital_rese_ps           168179 non-null  float64
 8    surplus_rese_ps           166270 non-null  float64
 9    undist_profit_ps          168663 non-null  float64
 10   extra_item                169362 non-null  float64
 11   profit_dedt               169361 non-null  float64
 12   gross_margin              170054 non-null  float64
 1

2025-06-11 09:20:52 - MyQuantProject - INFO - 体检报告生成完毕！
2025-06-11 09:20:52 - MyQuantProject - INFO - ================== 数据探索任务成功结束 ==================



                     count  mean      std         min    25%  50%   75%        max
eps              168658.00  0.37     1.11      -11.93   0.03 0.18  0.49      68.64
dt_eps           164950.00  0.37     1.09      -11.93   0.03 0.18  0.49      68.64
total_revenue_ps 168603.00  4.92    17.23       -1.36   1.06 2.46  5.33    5306.54
revenue_ps       168570.00  4.92    17.23       -1.36   1.06 2.46  5.32    5306.54
capital_rese_ps  168179.00  3.98   233.38      -14.82   0.95 1.98  3.69   67521.29
...                    ...   ...      ...         ...    ...  ...   ...        ...
tr_yoy           167037.00 33.76  1193.66     -155.23  -8.48 7.64 27.44  191312.33
or_yoy           167002.00 33.86  1193.81     -155.23  -8.50 7.65 27.48  191312.33
q_sales_yoy      159713.00 91.44  8417.77    -2473.34 -11.50 6.78 28.77 2484539.22
q_op_qoq         167530.00 48.06 11952.17 -1445158.47 -51.72 1.19 77.43 2015261.46
equity_yoy       159911.00 17.40   498.12   -52263.29   0.26 5.82 15.80   80243.11

[10